# Basic text recognition

In this tutorial I will explore the basics for text recognition with tensorflow, following one of the basic Keras tutorials. The exercise will be to use the Large Movie Review Dataset (a subset of IMDB) to perform sentiment analysis. Sentiments are divided into two categories: positive and negative. As such, this will be a binary classification problem.

The training and testing sets are balanced, _i.e._ they contain an equal number of positive and negative reviews.

In [2]:
import matplotlib.pyplot as plt
import os
import re
import shutil 
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses

In [3]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1", url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

In [4]:
os.listdir(dataset_dir)

['imdb.vocab', 'imdbEr.txt', 'README', 'test', 'train']

In [5]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['labeledBow.feat',
 'neg',
 'pos',
 'unsup',
 'unsupBow.feat',
 'urls_neg.txt',
 'urls_pos.txt',
 'urls_unsup.txt']

Directories 'pos' and 'neg' contain many text files, let's see one of them.

In [14]:
sample_file = os.path.join(train_dir, 'pos/1181_9.txt')
with open(sample_file) as f:
    print(f.read())

Rachel Griffiths writes and directs this award winning short film. A heartwarming story about coping with grief and cherishing the memory of those we've loved and lost. Although, only 15 minutes long, Griffiths manages to capture so much emotion and truth onto film in the short space of time. Bud Tingwell gives a touching performance as Will, a widower struggling to cope with his wife's death. Will is confronted by the harsh reality of loneliness and helplessness as he proceeds to take care of Ruth's pet cow, Tulip. The film displays the grief and responsibility one feels for those they have loved and lost. Good cinematography, great direction, and superbly acted. It will bring tears to all those who have lost a loved one, and survived.


## Load dataset
We need to load the data off disk and prepare it into a format suitable for training. We can use the 'text_dataset_from_directory' utility. 

We need two folders on disk, corresponding to class_a and class_b. We will thus remove the additional folders from this dataset.

In [16]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

Next, we use the aforementioned utility to create a labeled tf.data.Dataset object. We split the dataset into three splits: train, validation and test. In the IMDB dataset we don't have the validation set, so we create it using a 80:20 split.

In [30]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train/',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


We can also iterate over the dataset and print out a few examples:

In [31]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(3):
        print("Review", text_batch.numpy()[i])
        print("Label", label_batch.numpy()[i])

Review b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
Label 0
Review b"David Mamet is a very interesting and a very un-equal director. His first movie 'House of Games' was the one I liked best, and it set a series of films with characters whose perspective of life changes as they get into 

The reviews contain raw text and occasional HTML tags. We will see later how to deal with those. Let's see what the labels 0 and 1 correspond to:

In [32]:
print("Label 0 corresponds to", raw_train_ds.class_names[0])
print("Label 1 corresponds to", raw_train_ds.class_names[1])

Label 0 corresponds to neg
Label 1 corresponds to pos


Next we prepare validation and test datasets:

In [35]:
raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train/',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed
)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [36]:
raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test/',
    batch_size=batch_size
)

Found 25000 files belonging to 2 classes.


### Prepare dataset for training